
# USB Webcam Tutorial (V4L2 + OpenCV, Linux/Ubuntu)

This hands-on notebook shows how to:
- Discover USB webcams and inspect capabilities with **V4L2** (`v4l2-ctl`).
- Configure formats (e.g., **MJPEG** or **YUYV**), resolution, FPS, and camera controls (exposure, focus, WB).
- Capture images and video with **OpenCV** (both windowed & headless modes).
- Benchmark FPS, handle MJPEG/YUYV, and record to MP4.

> **Tested environment**: Ubuntu 22.04/24.04, Python 3.10+, OpenCV 4.x, a UVC-compatible USB webcam.
>
> **Prerequisites** (run once):
> ```bash
> sudo apt update
> sudo apt install -y v4l-utils
> pip install opencv-python
> ```


## 0. Environment Check

In [ ]:

import sys, platform, subprocess, shutil, os, time, re, json, glob, pathlib
import cv2
from IPython.display import display, Markdown, clear_output

print("Python:", sys.version)
print("Platform:", platform.platform())
print("OpenCV:", cv2.__version__)

# Check v4l2-ctl availability
v4l2_path = shutil.which("v4l2-ctl")
print("v4l2-ctl:", v4l2_path if v4l2_path else "NOT FOUND - please `sudo apt install v4l-utils`")


## 1. Discover Video Devices

In [ ]:

# List /dev/video* nodes
video_nodes = sorted(glob.glob("/dev/video*"))
print("Detected video nodes:", video_nodes)

# v4l2-ctl --list-devices gives a nice mapping (device -> /dev/videoX)
if v4l2_path:
    print("\n== v4l2-ctl --list-devices ==")
    print(subprocess.run(["v4l2-ctl", "--list-devices"], capture_output=True, text=True).stdout)
else:
    print("v4l2-ctl not available; skipping device listing via v4l2-ctl.")



## 2. Choose Your Webcam Device

Set `DEVICE` to the `/dev/videoX` node of your webcam. If you're unsure, pick the first one that shows UVC capabilities in the previous step.


In [ ]:

# Change this to your webcam node if needed
DEVICE = "/dev/video0"
DEVICE


## 3. Inspect Capabilities, Formats, and Frame Sizes

In [ ]:

if not os.path.exists(DEVICE):
    raise FileNotFoundError(f"{DEVICE} not found. Update DEVICE to a valid /dev/videoX.")

if v4l2_path:
    print("== v4l2-ctl --device --all ==")
    print(subprocess.run(["v4l2-ctl", f"--device={DEVICE}", "--all"], capture_output=True, text=True).stdout)

    print("\n== v4l2-ctl --device --list-formats-ext ==")
    print(subprocess.run(["v4l2-ctl", f"--device={DEVICE}", "--list-formats-ext"], capture_output=True, text=True).stdout)
else:
    print("v4l2-ctl not available; cannot show capabilities/formats.")



## 4. (Optional) Configure Format & FPS via V4L2

Two common pixel formats:
- **MJPG** (Motion JPEG): lower USB bandwidth, lighter CPU decode than raw → often best for 1080p+ over USB.
- **YUYV** (YUYV 4:2:2): raw frames, higher bandwidth but low latency and no compression artifacts.

> We'll try setting **1920x1080 @ 30fps** with **MJPG**. Adjust if unsupported by your camera.


In [ ]:

PREFERRED_WIDTH, PREFERRED_HEIGHT, PREFERRED_FPS = 1920, 1080, 30
PREFERRED_FOURCC = "MJPG"  # or "YUYV"

if v4l2_path:
    print("Setting format via v4l2-ctl ...")
    cmds = [
        ["v4l2-ctl", f"--device={DEVICE}", f"--set-fmt-video=width={PREFERRED_WIDTH},height={PREFERRED_HEIGHT},pixelformat={PREFERRED_FOURCC}"],
        ["v4l2-ctl", f"--device={DEVICE}", f"--set-parm={PREFERRED_FPS}"]
    ]
    for c in cmds:
        res = subprocess.run(c, capture_output=True, text=True)
        print("$", " ".join(c))
        if res.stderr.strip():
            print("stderr:", res.stderr.strip())
        if res.stdout.strip():
            print(res.stdout.strip())
else:
    print("v4l2-ctl not available; skip format set.")



## 5. OpenCV Capture Basics

We'll show two patterns:

- **Headless (Notebook)**: display a few frames inline (no GUI windows).
- **Windowed (Desktop)**: show a live window. Use this on a local desktop with a display server.


In [ ]:

def open_capture(dev="/dev/video0", width=1280, height=720, fps=30, fourcc="MJPG"):
    cap = cv2.VideoCapture(dev, cv2.CAP_V4L2)  # prefer V4L2 backend on Linux
    if fourcc:
        # set FOURCC before size/fps for reliability
        cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*fourcc))
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    cap.set(cv2.CAP_PROP_FPS, fps)
    # Some drivers report after opening; query back
    actual = {
        "fourcc": int(cap.get(cv2.CAP_PROP_FOURCC)),
        "width": int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        "height": int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
        "fps": cap.get(cv2.CAP_PROP_FPS),
        "backend": int(cap.get(cv2.CAP_PROP_BACKEND))
    }
    return cap, actual

cap, actual = open_capture(DEVICE, PREFERRED_WIDTH, PREFERRED_HEIGHT, PREFERRED_FPS, PREFERRED_FOURCC)
print("Actual settings:", actual)
if not cap.isOpened():
    raise RuntimeError("Failed to open the camera. Check permissions and device node.")


### 5.1 Headless Preview (Inline Frames)

In [ ]:

import numpy as np
from IPython.display import display
import ipywidgets as widgets

n_frames = 5
imgs = []
for i in range(n_frames):
    ok, frame = cap.read()
    if not ok:
        print("Failed to read frame")
        break
    # Optional: convert color if needed (OpenCV default is BGR)
    # display inline
    _, buf = cv2.imencode(".jpg", frame)
    display(Markdown(f"**Frame {i+1}**"))
    display(widgets.Image(value=buf.tobytes(), format='jpg', width=512))

# Keep cap open for subsequent cells


### 5.2 Save a Snapshot

In [ ]:

out_path = "snapshot.jpg"
ok, frame = cap.read()
if ok:
    cv2.imwrite(out_path, frame)
    print("Saved:", out_path, os.path.abspath(out_path))
else:
    print("Failed to grab a frame for snapshot.")



### 5.3 (Optional) Windowed Live View

> Run this only on a local desktop with a display (won't work on headless servers). Press **q** to exit.


In [ ]:

# Uncomment to use in a non-notebook Python session with a display
import cv2, time
win = "Live"
cv2.namedWindow(win, cv2.WINDOW_NORMAL)
while True:
    ok, frame = cap.read()
    if not ok:
        break
    cv2.imshow(win, frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


## 6. Handling YUYV (Raw 4:2:2)

In [ ]:

# If MJPG is unavailable or you prefer raw frames, try YUYV.
# We'll reopen with YUYV to demonstrate conversion.

cap.release()
cap, actual = open_capture(DEVICE, 640, 480, 30, "YUYV")
print("Reopened with YUYV. Actual:", actual)

ok, frame = cap.read()
if not ok:
    print("Failed to read YUYV frame; your camera/driver may not support raw at this size/fps.")
else:
    # Some backends already convert to BGR; if you get a single channel or strange shape, use cvtColor:
    # Example: yuyv_bgr = cv2.cvtColor(frame, cv2.COLOR_YUV2BGR_YUY2)
    # For demo, we will just save whatever we get:
    cv2.imwrite("yuyv_sample.jpg", frame)
    print("Saved YUYV sample as decoded by backend:", os.path.abspath("yuyv_sample.jpg"))


## 7. Record to MP4 (H.264 or MPEG-4 Part 2)

In [ ]:

# We'll use 'mp4v' to guarantee broad compatibility. For hardware encoders, prefer GStreamer or FFmpeg pipelines.
cap.release()
cap, actual = open_capture(DEVICE, 1280, 720, 30, "MJPG")
print("Recording with:", actual)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # alternatively "avc1" if supported
out = cv2.VideoWriter("capture.mp4", fourcc, 30, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

frames = 150  # ~5 seconds @30fps
grabbed = 0
for _ in range(frames):
    ok, frame = cap.read()
    if not ok:
        break
    out.write(frame)
    grabbed += 1

out.release()
cap.release()
print(f"Wrote {grabbed} frames to", os.path.abspath("capture.mp4"))


## 8. Camera Controls (Exposure, Focus, White Balance, etc.)

In [ ]:

if v4l2_path:
    print("== List all user controls ==")
    print(subprocess.run(["v4l2-ctl", f"--device={DEVICE}", "--list-ctrls-menus"], capture_output=True, text=True).stdout)

    # Examples (uncomment what you need):
    # Turn off auto exposure and set manual exposure time:
    # subprocess.run(["v4l2-ctl", f"--device={DEVICE}", "--set-ctrl=exposure_auto=1"], check=False)           # 1=Manual Mode for many UVC cams
    # subprocess.run(["v4l2-ctl", f"--device={DEVICE}", "--set-ctrl=exposure_absolute=200"], check=False)     # value depends on camera

    # Disable auto white balance and set temperature:
    # subprocess.run(["v4l2-ctl", f"--device={DEVICE}", "--set-ctrl=white_balance_temperature_auto=0"], check=False)
    # subprocess.run(["v4l2-ctl", f"--device={DEVICE}", "--set-ctrl=white_balance_temperature=5000"], check=False)

    # Disable auto focus (if supported) and set absolute focus:
    # subprocess.run(["v4l2-ctl", f"--device={DEVICE}", "--set-ctrl=focus_auto=0"], check=False)
    # subprocess.run(["v4l2-ctl", f"--device={DEVICE}", "--set-ctrl=focus_absolute=10"], check=False)

    # Brightness/Contrast/Gain (names vary by camera):
    # subprocess.run(["v4l2-ctl", f"--device={DEVICE}", "--set-ctrl=brightness=128"], check=False)
    # subprocess.run(["v4l2-ctl", f"--device={DEVICE}", "--set-ctrl=contrast=32"], check=False)
    # subprocess.run(["v4l2-ctl", f"--device={DEVICE}", "--set-ctrl=gain=0"], check=False)
else:
    print("v4l2-ctl not available; cannot show/set controls.")


## 9. Simple FPS Benchmark

In [ ]:

import time

def benchmark_fps(dev, width=1280, height=720, fps=30, fourcc="MJPG", seconds=5):
    cap, actual = open_capture(dev, width, height, fps, fourcc)
    if not cap.isOpened():
        raise RuntimeError("Cannot open device for benchmark.")
    count = 0
    t0 = time.time()
    while time.time() - t0 < seconds:
        ok, frame = cap.read()
        if not ok:
            break
        count += 1
    cap.release()
    elapsed = time.time() - t0
    return count / elapsed if elapsed > 0 else 0, actual

measured_fps, actual = benchmark_fps(DEVICE, 1280, 720, 30, "MJPG", 5)
print(f"Measured ~{measured_fps:.1f} FPS with settings:", actual)



## 10. Best Practices & Troubleshooting

**Best Practices**
1. Prefer **MJPG** for high resolutions over USB (reduced bandwidth). Use **YUYV** if you need raw frames or ultra-low latency.
2. Set the **FOURCC before width/height/fps** in OpenCV for more reliable negotiation.
3. Lock **auto-exposure/auto-WB/auto-focus** for consistent results (use `v4l2-ctl --set-ctrl=...`).
4. Stick to **standard modes** (e.g., 640×480, 1280×720, 1920×1080 @ 30 FPS) that your webcam lists via `--list-formats-ext`.
5. For headless servers, disable OpenCV windows and use **inline display** (JPEG) or just save to files.
6. When recording MP4, use `mp4v` for compatibility. For **hardware encoding**, consider FFmpeg/GStreamer pipelines.

**Troubleshooting**
- **Device not found**: Check `ls -l /dev/video*`, USB permissions, and group membership (e.g., add your user to `video` group).
- **Cannot set FPS/resolution**: The camera may not support that combo; check `--list-formats-ext` and pick a supported tuple.
- **Color looks wrong**: If using YUYV, ensure proper `cvtColor` (e.g., `COLOR_YUV2BGR_YUY2`). Some backends auto-convert already.
- **Low FPS**: Try MJPG; avoid USB hubs; use a USB 3.0 port; reduce resolution; close other apps using the camera.
- **OpenCV backend issues**: Force `cv2.CAP_V4L2` on Linux; verify OpenCV build includes V4L2 backend.
- **Permission denied**: Run `sudo udevadm control --reload-rules && sudo udevadm trigger` after adjusting udev rules, or run the notebook as a user in the `video` group.



## 11. Appendix: Minimal One-Shot Examples

**Capture one JPEG (MJPG path)**
```python
import cv2
cap = cv2.VideoCapture("/dev/video0", cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
cap.set(cv2.CAP_PROP_FPS, 30)
ok, frame = cap.read()
if ok: cv2.imwrite("one_shot.jpg", frame)
cap.release()
```

**Use YUYV and convert**
```python
import cv2
cap = cv2.VideoCapture("/dev/video0", cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"YUYV"))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 30)
ok, yuyv = cap.read()
if ok:
    # Some backends already convert; if needed:
    bgr = cv2.cvtColor(yuyv, cv2.COLOR_YUV2BGR_YUY2)
    cv2.imwrite("yuyv_one_shot.jpg", bgr)
cap.release()
```

**Record 10 seconds to MP4**
```python
import cv2, time
cap = cv2.VideoCapture("/dev/video0", cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_FPS, 30)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("out.mp4", fourcc, 30, (1280, 720))
t0 = time.time()
while time.time() - t0 < 10:
    ok, frame = cap.read()
    if not ok: break
    out.write(frame)
out.release(); cap.release()
```
